## Description
The objective of this competition is to create a machine learning model to detect fraudulent transactions.

Fraud detection is an important application of machine learning in the financial services sector. This solution will help Xente provide improved and safer service to its customers.

This competition is sponsored by Xente, Innovation Village, and insight2impact.

## Data
Xente is an e-commerce and financial service app serving 10,000+ customers in Uganda.

This dataset includes a sample of approximately 140,000 transactions that occurred between 15 November 2018 and 15 March 2019.

One of the challenges of fraud detection problems is that the data is highly imbalanced. 

Xente_variable_definitions.csv: Definition of the features per transaction
Training.csv: Transactions from 15 November 2018 to 13 February 2019, including whether or not each transaction is fraudulent. You will use this file to train your model.
Test.csv: Transactions from 13 February 2019 to 14 March 2019, not including whether or not each transaction is fraudulent. You will test your model on this file.
sample_submission.csv: is an example of what your submission file should look like. The order of the rows does not matter, but the names of the TransactionId must be correct. The value in FraudResult will be 1 for is a Fraud and 0 for is not a fraud.

## Evaluation
The error metric for this competition is the F1 score, which ranges from 0 (total failure) to 1 (perfect score). Hence, the closer your score is to 1, the better your model.

F1 Score: A performance score that combines both precision and recall. It is a harmonic mean of these two variables. Formula is given as: 2*Precision*Recall/(Precision + Recall)

Precision: This is an indicator of the number of items correctly identified as positive out of total items identified as positive. Formula is given as: TP/(TP+FP)

Recall / Sensitivity / True Positive Rate (TPR): This is an indicator of the number of items correctly identified as positive out of total actual positives. Formula is given as: TP/(TP+FN)

Where:

TP=True Positive
FP=False Positive
TN=True Negative
FN=False Negative

Info from Leaderboard: score to beat: 0,89

In [ ]:
import pandas as pd
from datetime import datetime, date, time, timedelta
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!pip install plotly

In [ ]:
# Load the variable names
variable_meanings = pd.read_csv("data/variable meanings.csv")
pd.set_option('max_colwidth', 800)
variable_meanings

## Results from Pandas Profile
1. There is only one `CurrencyCode`, that means there is no additional information provided by this feature ==> Drop CurrencyCode
2. There is only one CountryCode, that means there is no additional information provided by this feature ==> Drop CountryCode
3. TransactionIDs are all distinct, that means there is no additional information provided ==> Drop TransactionIds
4. TransactionStartTime consist of timestamps. For further analysis ==> group them into timeframes (use "datetime")
5. Amount contains + and - values (due to debit/credit) ==> we need to create a column with debit credit and transform "amount" to absolut values
6. Extreeeemely imbalanced target value ==> Oversampling? ==> read further information / links auf zindi nutzen
7. Definition for column  CustomerID and AccountId seems to be mixed up

8. Transform variable amount (log) due to skewness ==> no log function for negative values ==> we won't use this variable in model
9. Transform variable value or value (USD) (log) due to skewness => we choose ValueUSD (smaller values)
(10.Transform variable Providerid (log) due to skewness)


In [ ]:
# Load the data
data_test = pd.read_csv("data/test.csv")
data_train = pd.read_csv("data/training.csv")


In [ ]:
data_train.head()

In [ ]:
data_train.shape

---

## Data cleaning

* Stripped the ID columns from non-integer characters and converted them to integers
* Separated TransactionStartTime into transactiontime and transactiondate
* Drop redundant columns

In [ ]:
def remove_letters(string):
    return int(string.split('_')[1])
    
id_columns = ["TransactionId","BatchId","AccountId","SubscriptionId","CustomerId","ProviderId","ProductId","ChannelId"]    
for i in id_columns:
    data_train[i] = data_train[i].apply(lambda x:remove_letters(x))

In [ ]:
data_train.head()

In [ ]:
# seperate `TransactionStartTime` into time and date
def convert_to_date(date):
    # convert field into datetime format
    d = datetime.strptime(date,'%Y-%m-%dT%H:%M:%SZ')
    # extract date
    return d.date()

def convert_to_time(date):
    d = datetime.strptime(date,'%Y-%m-%dT%H:%M:%SZ')
    # extract time
    return d.time()

# create new columns with seperate information for `TransactionTime` and `TransactionDate`
data_train['TransactionTime'] = data_train.TransactionStartTime.apply(lambda x: convert_to_time(x))
data_train['TransactionDate'] = data_train.TransactionStartTime.apply(lambda x: convert_to_date(x))

Consolidate times into seperate blocks:

1. 00:00 - 05:59 (night)
2. 06:00 - 09:59 (morning)
3. 10:00 - 13:59 (midday)
4. 14:00 - 17:59 (afternoon)
5. 18:00 - 23:59 (evening)

In [ ]:
# verify that time scale is 0-23
#data_train.TransactionTime.apply(lambda x: x.hour).value_counts()

In [ ]:
# apply day time consolidation
def consolidate_time(time):
    if time.hour < 6:
        return 'night'
    elif time.hour < 10:
        return 'morning'
    elif time.hour < 14:
        return 'midday'
    elif time.hour < 18:
        return 'afternoon'
    else:
        return 'evening'
    
data_train['DayTime'] = data_train.TransactionTime.apply(lambda x: consolidate_time(x))

In [ ]:
# extract weekdays from `TransactionDate`
data_train['TransactionWeekday'] = data_train.TransactionDate.apply(lambda x: x.isoweekday())

In [ ]:
# create new feature to distinguish between Debit (0) and Credit (1)
data_train['DebitCredit'] = data_train.Amount.apply(lambda x: 0 if x > 0 else 1)

In [ ]:
# add column for value in USD (1 UGX = 0.00028 USD (12.01.2022, UTC 12:10))
data_train['ValueUSD'] = data_train.Value.apply(lambda x: x * 0.00028)

Überlegungen Feature Engineering:
    - Spalte einfügen mit Anzahl der Transaktionen je Batch ==> prüfen ob Auffälligkeit der Batchgröße bei Frauds
    - Spalte einfügen mit Amount != Value : Absolutbetrag Differenz nutzen
    - Spalte einfügen mit Anzahl der bis zum Zeitpunkt der aktuellen Transaktion (Transactions to date) getätigten Transaktionen je Account

In [ ]:
def convert_to_isoweek(date):
    return date.isocalendar()[1]

data_train['ISOWeek'] = data_train.TransactionDate.apply(lambda x: convert_to_isoweek(x))

In [ ]:
# 2. Step (after first models) create new feature Transaktion per Batch
a = data_train.groupby('BatchId', as_index=False)['TransactionId'].count()
#a.unique()
a.rename(columns= {'TransactionId': 'TransactionInBatch' }, inplace=True)
data_train = data_train.merge(a, on='BatchId')
data_train.head()

In [ ]:
# 2. Step (after first models) Difference Amount and Value 

data_train['value_amount_diff'] = abs(data_train["Value"] - data_train["Amount"])
data_train.head()

In [ ]:
# 2. Step (after first models) 
def convert_to_datetime(date):
    # convert field into datetime format
    d = datetime.strptime(date,'%Y-%m-%dT%H:%M:%SZ')
    # extract date
    return d
data_train['DT'] = data_train.TransactionStartTime.apply(lambda x: convert_to_datetime(x))

In [ ]:
# 2. Step (after first models) 
# only ONE account_id
def transactions_toDate(df, transaction_id, account_id):
    """
    returns dataframe
    """
    # create empty dictionary
    TTD = {'t_id': [], 'a_id': [], 
           'TransactionsToDate': []}#, 'date': []}

    count = 0
    # iterate through all transaction ids for one account id and assign counts
    for t in transaction_id:
        TTD['t_id'] += [t]
        TTD['a_id'] += [account_id]
        TTD['TransactionsToDate'] += [count]
        count += 1
    # return counts in data frame format 
    return pd.DataFrame.from_dict(TTD)

In [ ]:
temp = pd.DataFrame()
for i in data_train.AccountId.unique():
    df = data_train.query('AccountId == @i')
    
    # count seperately for every sub set of account ids
    TTD = transactions_toDate(df, df.TransactionId,i)
    # add counts vertically to temporary data frame
    temp = pd.concat([temp, TTD], axis=0)

In [ ]:
data_train = data_train.merge(temp, left_on='TransactionId', right_on='t_id')
data_train.drop(['t_id', 'a_id'], inplace=True, axis=1)

In [ ]:
Value_Amount = data_train.query('abs(Value) != abs(Amount) and FraudResult ==1')[['Value','Amount','ProductId','BatchId']]

In [ ]:
Value_Amount['dif']= abs(Value_Amount.Value - Value_Amount.Amount)
Value_Amount

In [ ]:
# 2565 rows with differences in Value/Amount ==> maybe drop these rows later

In [ ]:
#data_train.groupby('AccountId')['CustomerId'].count()
#sorted(data_train.groupby('CustomerId')['AccountId'].count(), reverse = True )

In [ ]:
data_train.query('AccountId != SubscriptionId')[['AccountId','SubscriptionId']]

==>SubscriptionId und AccountID sind immer unterschiedlich!

In [ ]:
data_train.query('AccountId == CustomerId')[['AccountId','CustomerId','FraudResult']]

In [ ]:
data_train.head()

In [ ]:
# drop columns that do not convey additional meaning
#'TransactionId' ==> remove later before modelling
cols_to_drop = ['CurrencyCode', 'CountryCode', 'TransactionStartTime', 'BatchId','DT','TransactionTime','Amount','ValueUSD','TransactionDate']
data_train_clean = data_train.drop(columns=cols_to_drop, inplace=False)
data_train_clean.to_csv('data/data_train_clean.csv')

In [ ]:
data_train_clean.head()

In [ ]:
fraudparts= data_train_clean.groupby('FraudResult')['TransactionId'].count()
fraudparts

In [ ]:
fraudcosts= data_train_clean.groupby('FraudResult')['Value'].sum()
fraudcosts

In [ ]:
debitcredit= data_train_clean.query('FraudResult == 1').groupby('DebitCredit')['Value'].sum()
debitcredit

In [ ]:
pivot1 = pd.pivot_table(data_train_clean, values='TransactionId', index='ProductId', columns='ProductCategory', aggfunc ='count')
pivot1.fillna('0', inplace = True)
pivot1

In [ ]:
pivot1 = pd.pivot_table(data_train_clean, values='Value', index='FraudResult', columns='ProductCategory', aggfunc =sum)
pivot1

- Columns Transactionweekday: Unauffällig
- Columns Daytime: Unauffällig

In [ ]:
pivot2 = pd.pivot_table(data_train_clean.query('FraudResult ==1'), values='TransactionId', index='PricingStrategy', columns='ProductId', aggfunc ='count')
pivot2.fillna('0', inplace = True)
pivot2

In [ ]:
pivot2a = pd.pivot_table(data_train_clean.query('FraudResult ==1'), values='Value', index='PricingStrategy', columns='ProductId', aggfunc ='sum')
pivot2a.fillna('0', inplace = True)
pivot2a


===> Häufung der Frauds bei Konstellation PricingStrategy=2 und ProductId=15

In [ ]:
pivot3 = pd.pivot_table(data_train.query('abs(Value) != abs(Amount)'), values='TransactionId', index='FraudResult', columns='ProductId', aggfunc ='count')
pivot3

Überlegungen Feature Engineering:
    - Spalte einfügen mit Anzahl der Transaktionen je Batch ==> prüfen ob Auffälligkeit der Batchgröße bei Frauds
    - Spalte einfügen mit Amount != Value : Absolutbetrag differenz nutzen
    - Spalte einfügen mit Anzahl der bis zum Zeitpunkt der aktuellen Transaktion (Transactions to date) getätigten Transaktionen je Account

In [ ]:
pivot4 = pd.pivot_table(data_train.query('FraudResult ==1'), values='TransactionId', index='TransactionInBatch', columns='FraudResult', aggfunc ='count')
pivot4 = pd.DataFrame(pivot4)
pivot4

In [ ]:
ax = pivot4.T.plot(kind='bar', ylabel='Amount frauds', xlabel= 'Transaction per Batch',title='Transactions per Batch (frauds)',color=['darkcyan','darkslategrey','cyan'])
#plt.xticks( [1,12,4] );


In [ ]:
pivot6 = pd.pivot_table(data_train.query('FraudResult ==1'), values='TransactionId', index='AccountId', columns='FraudResult', aggfunc ='count')
pivot6 = pivot6[1].sort_values(ascending = False)
pivot6

In [ ]:
pivot5 = pd.pivot_table(data_train.query('FraudResult ==1'), values='TransactionId', index='TransactionsToDate', columns='FraudResult', aggfunc ='count')
pivot5 = pd.DataFrame(pivot5)


### Data Visualization

In [ ]:
px.pie(data_train_clean.groupby('DebitCredit').count()[['TransactionId']].reset_index(), values = 'TransactionId', names ='DebitCredit',title='Percentage of Debit and Credit',color_discrete_sequence=px.colors.qualitative.Prism )

### Transform Data


9. Transform variable value or value (USD) (log) due to skewness
(10.Transform variable Providerid (log) due to skewness)

In [ ]:
data_train_clean['ValueLog']=np.log(data_train_clean.Value)

### Dataframe for model

In [ ]:
data_train_clean.columns

In [ ]:
cols_to_drop = ['Value','TransactionId', 'SubscriptionId']
df = data_train_clean.drop(columns=cols_to_drop, inplace=False)

In [ ]:
df = pd.get_dummies(df, columns = ['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'PricingStrategy', 'DayTime','TransactionWeekday','DebitCredit'], drop_first=True)


In [ ]:
df.to_csv('data/data_train_clean_withdummies.csv', index = False)